In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import scanpy as sc
import sys

import os 
import sys
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix

from nicheformer.data.constants import DefaultPaths, ObsConstants, UnsConstants, VarConstants, AssayOntologyTermId, SexOntologyTermId, OrganismOntologyTermId, TissueOntologyTermId, SuspensionTypeId
from nicheformer.data.validate import validate

In [26]:
import sys
sys.path.append("/home/ec2-user/SageMaker/functions")
from preprocessing import *


In [27]:
#if len(sys.argv)==1:
path = "./our_data_LC_NE"
#else:
#    path = sys.argv[1]

# raw_path = f"{path}"
# preprocessed_path = f"{path}/preprocessed"

# if not os.path.exists(f"{preprocessed_path}"):
#     os.mkdir(f"{preprocessed_path}")

adata = sc.read_h5ad(f"{path}/adata_mer.h5ad")


# setting MERFISH for now until an official ontology term is released
assay = str(AssayOntologyTermId.MERFISH_SPATIAL.value)
sex = str(SexOntologyTermId.UNKNOWN.value)
organism = str(OrganismOntologyTermId.MOUSE.value)
organism_validator = "mouse"
tissue = str(TissueOntologyTermId.BRAIN.value)
suspension_type = str(SuspensionTypeId.SPATIAL.value)
tissue_type = "brain"




In [28]:
adata

AnnData object with n_obs × n_vars = 3227 × 315
    obs: 'slicename', 'x_CCF', 'y_CCF', 'z_CCF', 'mouse_name', '_scvi_batch', '_scvi_labels', 'leiden', 'n_counts'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'leiden', 'leiden_colors', 'log1p', 'mouse_name_colors', 'neighbors', 'slicename_colors', 'umap'
    obsm: 'X_scVI', 'X_umap', 'spatial'
    obsp: 'connectivities', 'distances'

In [29]:
#adata.X = csr_matrix(adata.X)
adata.obs[ObsConstants.SPATIAL_X] = adata.obs['z_CCF']
adata.obs[ObsConstants.SPATIAL_Y] = adata.obs['y_CCF']
adata.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical([assay for i in range(len(adata))])
adata.obs[ObsConstants.SEX_ONTOLOGY_TERM_ID] = pd.Categorical([sex for i in range(len(adata))])
adata.obs[ObsConstants.ORGANISM_ONTOLOGY_TERM_ID] = pd.Categorical([organism for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_ONTOLOGY_TERM_ID] = pd.Categorical([tissue for i in range(len(adata))])
adata.obs[ObsConstants.SUSPENSION_TYPE] = pd.Categorical([suspension_type for i in range(len(adata))])

adata.obs[ObsConstants.DONOR_ID] = pd.Categorical(['Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs' for i in range(len(adata))])
adata.obs[ObsConstants.CONDITION_ID] = pd.Categorical(['non diseased' for i in range(len(adata))])
adata.obs[ObsConstants.TISSUE_TYPE] = pd.Categorical([tissue_type for i in range(len(adata))])

adata.uns[UnsConstants.TITLE] = "Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs"


adata.obs[ObsConstants.LIBRARY_KEY] = pd.Categorical(['section' for i in range(len(adata))])

In [30]:
adata.var

""
8030451A03Rik
9130024F11Rik
9630002D21Rik
Abca13
Abi3bp
...
Npy5r
Nr2f2
Nr4a2
Pdzrn3


In [31]:
adata_1 = ensem_for_adata(adata)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/ec2-user/.cache/pyensembl/GRCm39/ensembl104/Mus_musculus.GRCm39.pep.all.fa.gz.pickle


total invalid 1, from 315 0.31746031746031744 %
new data size (3227, 314)


/home/ec2-user/SageMaker/functions/preprocessing.py:28: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata_1.raw = adata_1.copy()


In [34]:
adata_1.var[VarConstants.FEATURE_IS_FILTERED] = False

In [36]:
adata.raw.X

array([[ 0.,  0.,  0., ...,  7.,  0.,  0.],
       [ 0.,  0.,  2., ...,  3.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ...,
       [ 0.,  0.,  3., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  3.,  0.,  0.]])

In [37]:
adata_1.raw.X

array([[0.        , 0.        , 0.        , ..., 8.88220375, 0.        ,
        0.        ],
       [0.        , 0.        , 8.0506384 , ..., 8.45599721, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 8.94044423, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 7.3397006 , ..., 8.54321883, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 9.4129463 , 0.        ,
        0.        ]])

In [35]:

adata_output, valid, errors, is_seurat_convertible = validate(adata_1, organism=organism_validator)

INFO:cellxgene_schema.validate:Starting validation...
ERROR:cellxgene_schema.validate:ERROR: Raw data may be missing: data in 'raw.X' contains non-integer values.
INFO:cellxgene_schema.validate:Validation complete in 0:00:00.517325 with status is_valid=False


In [21]:
adata_1.raw.var

,feature_is_filtered
ENSMUSG00000073821,False
ENSMUSG00000087022,False
ENSMUSG00000114028,False
ENSMUSG00000004668,False
ENSMUSG00000035258,False
...,...
ENSMUSG00000044014,False
ENSMUSG00000030551,False
ENSMUSG00000026826,False
ENSMUSG00000035357,False


In [20]:
adata_1.raw.var = adata_1.raw.var.index

AttributeError: can't set attribute 'var'

In [21]:
adata_output.obs['assay'] = pd.Categorical(['Xenium' for i in range(len(adata_output))])
adata_output.obs[ObsConstants.ASSAY_ONTOLOGY_TERM_ID] = pd.Categorical(['no yet defined' for i in range(len(adata_output))])

adata_output.obs[ObsConstants.DATASET] = adata_output.uns['title']
adata_output.obs[ObsConstants.SPLIT] = 'train'
adata_output.obs[ObsConstants.NICHE] = 'nan'
adata_output.obs[ObsConstants.REGION] = 'nan'

adata_output.write(f"{preprocessed_path}/Xenium_Preview_Human_Non_diseased_Lung_With_Add_on_FFPE_outs.h5ad")

TypeError: object of type 'NoneType' has no len()

In [32]:
adata.X.max()

66.0

In [24]:
adata_output